In [1]:
import requests
import json

In [2]:
A=requests.get('https://opencampus-codex.blockscout.com/api/v2/smart-contracts/0x49D8d8a61A7807a8cC78b42A34FA223014518863')
B=requests.get('https://opencampus-codex.blockscout.com/api/v2/smart-contracts/0x394c399dbA25B99Ab7708EdB505d755B3aa29997')


In [3]:
json.loads(A.content)["message"]

'Not found'

In [26]:
import uuid
def isContract(hash):
    A=requests.get('https://opencampus-codex.blockscout.com/api/v2/smart-contracts/%s'%(hash))
    if A.status_code==200:
        return True
    else:
        return False

def maxValue(num):
    if num<1:
        num=1
    if num > 20:
        num=20
    return int(num)

def jsonTx(Tx):
    AAA=Tx 
    if  'contract_creation' in AAA['tx_types']:
        try:
            Sal={
                "value":float(AAA["value"])/10**18,
                "width":maxValue(float(AAA["value"])/10**18),
                "fee":float(AAA['fee']['value'])/10**18,
                'tx_types':AAA['tx_types'],
                'timestamp':AAA['timestamp'],
                'block':AAA['block'],
                'token_transfers':AAA['token_transfers'],
                "from_ens":AAA['from']['ens_domain_name'],
                "from_wallet":AAA['from']['hash'],
                "from_iscontract":AAA['from']['is_contract'],
                "to_wallet":AAA['created_contract']['hash'],  
                "id" : str(uuid.uuid4())  
            }
        except:
            print("Fail")
        return Sal
    else:
        try:  
            Sal={
            "value":float(AAA["value"])/10**18,
            "fee":float(AAA['fee']['value'])/10**18,
            "width":maxValue(float(AAA["value"])/10**18),
            'tx_types':AAA['tx_types'],
            'timestamp':AAA['timestamp'],
            'block':AAA['block'],
            'token_transfers':AAA['token_transfers'],
            "to_ens":AAA['to']['ens_domain_name'],
            "to_wallet":AAA['to']['hash'],
            "to_iscontract":AAA['to']['is_contract'],
            "from_ens":AAA['from']['ens_domain_name'],
            "from_wallet":AAA['from']['hash'],
            "from_iscontract":AAA['from']['is_contract'],
            "id" : str(uuid.uuid4())
            }
        except:
            print("Fail")
        return Sal

def AllTx(Txs,Out):
    wallets=set()
    for i in json.loads(Txs)['items']:
        Temp=jsonTx(i)
        Out.append(Temp)
        wallets.add(Temp["from_wallet"])
        wallets.add(Temp["to_wallet"])
    return Out,wallets


    
def TxsWallet(wallet):
    Contract=isContract(wallet)
    if Contract:
        return {"message":"Is contract"},0,0
    Out=[]
    towallet=requests.get('https://opencampus-codex.blockscout.com/api/v2/addresses/%s/transactions?filter=to'%(wallet))
    Out,walletsto=AllTx(towallet.content,Out)
    fromwallet=requests.get('https://opencampus-codex.blockscout.com/api/v2/addresses/%s/transactions?filter=from'%(wallet))
    Out,walletsfrom=AllTx(fromwallet.content,Out)
    walletsto.union(walletsfrom)

    return Out,walletsto.union(walletsfrom),Contract

def colorContract(test):
    if test:
        return '#FF5733'
    else:
        return "#000000"
def TxAllWallets(wallet,maxloop=1000):
    wallets=[]
    if isContract(wallet):
        return {"message":"Is contract"}
    Out,newwallets,contract=TxsWallet(wallet)
    wallets.append({"id":wallet,"url":"https://opencampus-codex.blockscout.com/address/"+wallet,"color" : "#27ae60"})
    print(wallets)
    counter=0
    Tot=len(newwallets)
    Doit=set()
    for aw in newwallets:
        counter=counter+1
        if counter>maxloop-1:
            break
        print("%s de %s"%(counter, Tot))
        Out2,temp,Contract=TxsWallet(aw)
        if aw not in Doit:
            wallets.append({"id":aw,"url":"https://opencampus-codex.blockscout.com/address/"+aw,"color" : colorContract(isContract(aw))})
            Doit.add(aw)
        if temp!=0:
            Out3=Out+Out2
            Out=Out3

    return Out, wallets




In [26]:
isContract("0xE18eA458f28792D90bb2063A0A792d51D310207c")

200


True

In [28]:
TxsWallet("0xE18eA458f28792D90bb2063A0A792d51D310207c")

({'message': 'Is contract'}, 0, 0)

In [29]:
TxsWallet("0x49D8d8a61A7807a8cC78b42A34FA223014518863")

([{'value': 2.5,
   'fee': 5.47694e-05,
   'width': 2,
   'tx_types': ['coin_transfer'],
   'timestamp': '2024-08-24T12:53:50.000000Z',
   'block': 36515,
   'token_transfers': None,
   'to_ens': None,
   'to_wallet': '0x49D8d8a61A7807a8cC78b42A34FA223014518863',
   'to_iscontract': False,
   'from_ens': None,
   'from_wallet': '0xa482AE7d753b33cB06Fd6872443E902cB6BEe592',
   'from_iscontract': False,
   'id': 'e6b653c9-2a85-4fd9-8595-246b9b7bf4eb'}],
 {'0x49D8d8a61A7807a8cC78b42A34FA223014518863',
  '0xa482AE7d753b33cB06Fd6872443E902cB6BEe592'},
 False)

In [62]:
TxsWallet("0x49D8d8a61A7807a8cC78b42A34FA223014518863")

[{'value': '2500000000000000000',
  'fee': '54769400000000',
  'tx_types': ['coin_transfer'],
  'timestamp': '2024-08-24T12:53:50.000000Z',
  'block': 36515,
  'token_transfers': None,
  'to_ens': None,
  'to_wallet': '0x49D8d8a61A7807a8cC78b42A34FA223014518863',
  'to_iscontract': False,
  'from_ens': None,
  'from_wallet': '0xa482AE7d753b33cB06Fd6872443E902cB6BEe592',
  'from_iscontract': False}]

In [67]:
2500000000000000000/10**18
54769400000000/10**18

5.47694e-05

In [71]:
A,B=TxsWallet("0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf")

In [72]:
A

[{'value': 20.0,
  'fee': 5.39465e-05,
  'tx_types': ['coin_transfer'],
  'timestamp': '2024-08-24T07:48:26.000000Z',
  'block': 35726,
  'token_transfers': None,
  'to_ens': None,
  'to_wallet': '0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf',
  'to_iscontract': False,
  'from_ens': None,
  'from_wallet': '0x253F91e579C5260059804e6ac9057C92aD8B86b7',
  'from_iscontract': False},
 {'value': 10.0,
  'fee': 5.3535e-05,
  'tx_types': ['coin_transfer'],
  'timestamp': '2024-08-20T06:18:30.000000Z',
  'block': 29613,
  'token_transfers': None,
  'to_ens': None,
  'to_wallet': '0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf',
  'to_iscontract': False,
  'from_ens': None,
  'from_wallet': '0x253F91e579C5260059804e6ac9057C92aD8B86b7',
  'from_iscontract': False},
 {'value': 0.5055432,
  'fee': 5.43579e-05,
  'tx_types': ['coin_transfer'],
  'timestamp': '2024-08-12T10:48:52.000000Z',
  'block': 22696,
  'token_transfers': None,
  'to_ens': None,
  'to_wallet': '0x907fC0C7E6b84F0229c13F57D413F72D33Ff3

In [30]:
Aw,Bw=TxAllWallets("0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf",20)

[{'id': '0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf', 'url': 'https://opencampus-codex.blockscout.com/address/0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf', 'color': '#27ae60'}]
1 de 46
2 de 46
3 de 46
4 de 46
5 de 46
6 de 46
7 de 46
8 de 46
9 de 46
10 de 46
11 de 46
12 de 46
13 de 46
14 de 46
15 de 46
16 de 46
17 de 46
18 de 46
19 de 46


In [31]:
Aw

[{'value': 20.0,
  'fee': 5.39465e-05,
  'width': 20,
  'tx_types': ['coin_transfer'],
  'timestamp': '2024-08-24T07:48:26.000000Z',
  'block': 35726,
  'token_transfers': None,
  'to_ens': None,
  'to_wallet': '0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf',
  'to_iscontract': False,
  'from_ens': None,
  'from_wallet': '0x253F91e579C5260059804e6ac9057C92aD8B86b7',
  'from_iscontract': False,
  'id': '352fc479-798f-4002-b03b-11839ddd20e1'},
 {'value': 10.0,
  'fee': 5.3535e-05,
  'width': 10,
  'tx_types': ['coin_transfer'],
  'timestamp': '2024-08-20T06:18:30.000000Z',
  'block': 29613,
  'token_transfers': None,
  'to_ens': None,
  'to_wallet': '0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf',
  'to_iscontract': False,
  'from_ens': None,
  'from_wallet': '0x253F91e579C5260059804e6ac9057C92aD8B86b7',
  'from_iscontract': False,
  'id': '0dda60f3-bc9a-41c1-a54a-07cd7173c229'},
 {'value': 0.5055432,
  'fee': 5.43579e-05,
  'width': 1,
  'tx_types': ['coin_transfer'],
  'timestamp': '2024-08-

In [32]:
Bw

[{'id': '0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf',
  'url': 'https://opencampus-codex.blockscout.com/address/0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf',
  'color': '#27ae60'},
 {'id': '0x3B8878cB5790872b142D3577dCb9c628BEc425c2',
  'url': 'https://opencampus-codex.blockscout.com/address/0x3B8878cB5790872b142D3577dCb9c628BEc425c2',
  'color': '#000000'},
 {'id': '0x0AeA6B0eC79A1060E1736d76e0c38E34459Eb130',
  'url': 'https://opencampus-codex.blockscout.com/address/0x0AeA6B0eC79A1060E1736d76e0c38E34459Eb130',
  'color': '#000000'},
 {'id': '0x35c37ccac79e46DB496d8b6D706057B649c8F7d9',
  'url': 'https://opencampus-codex.blockscout.com/address/0x35c37ccac79e46DB496d8b6D706057B649c8F7d9',
  'color': '#000000'},
 {'id': '0xb2F0f9141Ab3DDA0150093b31ffFE51530ef3fd7',
  'url': 'https://opencampus-codex.blockscout.com/address/0xb2F0f9141Ab3DDA0150093b31ffFE51530ef3fd7',
  'color': '#000000'},
 {'id': '0xdafC43D1d96F002b3203cAD27f3DFdF8191DbeC9',
  'url': 'https://opencampus-codex.blockscout.

In [67]:
def makeData(Aw,Bw):
    listWallet=set()
    Sal1=[]
    for walletT in Bw:
        Sal1.append( { "id": walletT["id"], "url": walletT["url"], "color": walletT["color"] } )
        listWallet.add(walletT["id"])
    Sal2=[]
    counter=1
    for TxT in Aw: 
        if TxT["from_wallet"] not in listWallet:
            Sal1.append({ "id": TxT["from_wallet"], "url": "https://opencampus-codex.blockscout.com/address/"+TxT["from_wallet"], "color": colorContract(isContract(TxT["from_wallet"])) } )
        if TxT["to_wallet"] not in listWallet:
            Sal1.append({ "id": TxT["to_wallet"], "url": "https://opencampus-codex.blockscout.com/address/"+TxT["to_wallet"], "color": colorContract(isContract(TxT["to_wallet"])) } )
        Sal2.append({" id": TxT["id"], "source": TxT["from_wallet"], "target": TxT["to_wallet"], "width":TxT["width"] } )
        print(counter)
        counter+=1
    Sal=Sal1+Sal2
    return Sal
Ad=makeData(Aw,Bw)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244


In [68]:
Ad


[{'id': '0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf',
  'url': 'https://opencampus-codex.blockscout.com/address/0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf',
  'color': '#27ae60'},
 {'id': '0x3B8878cB5790872b142D3577dCb9c628BEc425c2',
  'url': 'https://opencampus-codex.blockscout.com/address/0x3B8878cB5790872b142D3577dCb9c628BEc425c2',
  'color': '#000000'},
 {'id': '0x0AeA6B0eC79A1060E1736d76e0c38E34459Eb130',
  'url': 'https://opencampus-codex.blockscout.com/address/0x0AeA6B0eC79A1060E1736d76e0c38E34459Eb130',
  'color': '#000000'},
 {'id': '0x35c37ccac79e46DB496d8b6D706057B649c8F7d9',
  'url': 'https://opencampus-codex.blockscout.com/address/0x35c37ccac79e46DB496d8b6D706057B649c8F7d9',
  'color': '#000000'},
 {'id': '0xb2F0f9141Ab3DDA0150093b31ffFE51530ef3fd7',
  'url': 'https://opencampus-codex.blockscout.com/address/0xb2F0f9141Ab3DDA0150093b31ffFE51530ef3fd7',
  'color': '#000000'},
 {'id': '0xdafC43D1d96F002b3203cAD27f3DFdF8191DbeC9',
  'url': 'https://opencampus-codex.blockscout.

In [53]:
Aw

[{'value': 20.0,
  'fee': 5.39465e-05,
  'width': 20,
  'tx_types': ['coin_transfer'],
  'timestamp': '2024-08-24T07:48:26.000000Z',
  'block': 35726,
  'token_transfers': None,
  'to_ens': None,
  'to_wallet': '0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf',
  'to_iscontract': False,
  'from_ens': None,
  'from_wallet': '0x253F91e579C5260059804e6ac9057C92aD8B86b7',
  'from_iscontract': False,
  'id': '352fc479-798f-4002-b03b-11839ddd20e1'},
 {'value': 10.0,
  'fee': 5.3535e-05,
  'width': 10,
  'tx_types': ['coin_transfer'],
  'timestamp': '2024-08-20T06:18:30.000000Z',
  'block': 29613,
  'token_transfers': None,
  'to_ens': None,
  'to_wallet': '0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf',
  'to_iscontract': False,
  'from_ens': None,
  'from_wallet': '0x253F91e579C5260059804e6ac9057C92aD8B86b7',
  'from_iscontract': False,
  'id': '0dda60f3-bc9a-41c1-a54a-07cd7173c229'},
 {'value': 0.5055432,
  'fee': 5.43579e-05,
  'width': 1,
  'tx_types': ['coin_transfer'],
  'timestamp': '2024-08-

In [56]:
import pandas as pd
df=pd.DataFrame(Aw)
A=pd.pivot_table(df, values="value",index=["to_wallet"],aggfunc="count").sort_values(by=['value'],ascending=False)


In [69]:
Bw

[{'id': '0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf',
  'url': 'https://opencampus-codex.blockscout.com/address/0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf',
  'color': '#27ae60'},
 {'id': '0x3B8878cB5790872b142D3577dCb9c628BEc425c2',
  'url': 'https://opencampus-codex.blockscout.com/address/0x3B8878cB5790872b142D3577dCb9c628BEc425c2',
  'color': '#000000'},
 {'id': '0x0AeA6B0eC79A1060E1736d76e0c38E34459Eb130',
  'url': 'https://opencampus-codex.blockscout.com/address/0x0AeA6B0eC79A1060E1736d76e0c38E34459Eb130',
  'color': '#000000'},
 {'id': '0x35c37ccac79e46DB496d8b6D706057B649c8F7d9',
  'url': 'https://opencampus-codex.blockscout.com/address/0x35c37ccac79e46DB496d8b6D706057B649c8F7d9',
  'color': '#000000'},
 {'id': '0xb2F0f9141Ab3DDA0150093b31ffFE51530ef3fd7',
  'url': 'https://opencampus-codex.blockscout.com/address/0xb2F0f9141Ab3DDA0150093b31ffFE51530ef3fd7',
  'color': '#000000'},
 {'id': '0xdafC43D1d96F002b3203cAD27f3DFdF8191DbeC9',
  'url': 'https://opencampus-codex.blockscout.

In [65]:
A.iloc[0].name
int(A.iloc[0].value)


8

In [54]:
import pandas as pd
df=pd.DataFrame(Aw)

pd.pivot_table(df, values="value",index=["to_wallet"],aggfunc="sum").sort_values(by=['value'],ascending=False)
pd.pivot_table(df, values="value",index=["to_wallet"],aggfunc="count").sort_values(by=['value'],ascending=False)

pd.pivot_table(df, values="value",index=["from_wallet"],aggfunc="sum").sort_values(by=['value'],ascending=False)
pd.pivot_table(df, values="value",index=["from_wallet"],aggfunc="count").sort_values(by=['value'],ascending=False)


,value
to_wallet,
0xa482AE7d753b33cB06Fd6872443E902cB6BEe592,1600.720000
0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf,30.505543
0x0AeA6B0eC79A1060E1736d76e0c38E34459Eb130,4.562197
0x0cddE1982fdB95c93258011636B7d3095D52e9DB,3.000000
0x41B07c0a41B0f3CC9FeCd5FD57fA847E751AF17b,3.000000
...,...
0xe4F7CD9c30c73914b75844E90Ec1cd1cf07C7c63,0.000000
0xf45a17A1d7dd7F70f0980DC3A6F5D7C9A4E05A01,0.000000
0xf2B1f7bd1129E21180Ac3D45d407facFf78b37F4,0.000000


In [55]:
import pandas as pd
df=pd.DataFrame(Aw)
pd.pivot_table(df, values="value",index=["from_wallet"],aggfunc="sum").sort_values(by=['value'],ascending=False)


,value
from_wallet,
0xb9eB6d46F4749Ce80Dfe90Cc3af8719E4B9a5775,950.720000
0x253F91e579C5260059804e6ac9057C92aD8B86b7,680.000000
0xa482AE7d753b33cB06Fd6872443E902cB6BEe592,85.090682
0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf,27.001500
0x0AeA6B0eC79A1060E1736d76e0c38E34459Eb130,1.900000
0x0cddE1982fdB95c93258011636B7d3095D52e9DB,0.600000
0x764D671CBC4819A692F7B79cb8eBb79987405B8F,0.116065
0x2689Fc05d470ED14A5Ca03D8D7338E4d6B7754be,0.100000
0xdafC43D1d96F002b3203cAD27f3DFdF8191DbeC9,0.100000


In [52]:
import pandas as pd
df=pd.DataFrame(Aw)
pd.pivot_table(df, values="value",index=["from_wallet"],aggfunc="count").sort_values(by=['value'],ascending=False)


,value
from_wallet,
0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf,72
0xa482AE7d753b33cB06Fd6872443E902cB6BEe592,55
0x0AeA6B0eC79A1060E1736d76e0c38E34459Eb130,22
0x763C8EE1F3C022d376771940234812f66ebB30EF,15
0x108D6aE998ebC3EE0685Dcd7c5f048e3B6d5aB26,10
0x35c37ccac79e46DB496d8b6D706057B649c8F7d9,9
0x97E6Cb287c9Ec894bFeBc733Ddd15fE8ba335A35,9
0x3B8878cB5790872b142D3577dCb9c628BEc425c2,8
0x2689Fc05d470ED14A5Ca03D8D7338E4d6B7754be,7


In [20]:
Aw,Bw=TxAllWallets("0x0d1d649753155e2903e80b89201FFF09E238Eb3B")
Aw

1 de 6
2 de 6
3 de 6
4 de 6
5 de 6
6 de 6


[{'value': 0.1,
  'fee': 5.3535e-05,
  'tx_types': ['coin_transfer'],
  'timestamp': '2024-08-26T11:11:46.000000Z',
  'block': 39993,
  'token_transfers': None,
  'to_ens': None,
  'to_wallet': '0x0d1d649753155e2903e80b89201FFF09E238Eb3B',
  'to_iscontract': False,
  'from_ens': None,
  'from_wallet': '0x2ec8175015Bef5ad1C0BE1587C4A377bC083A2d8',
  'from_iscontract': False},
 {'value': 0.0,
  'fee': 0.0001221063,
  'tx_types': ['contract_call'],
  'timestamp': '2024-08-28T15:53:06.000000Z',
  'block': 44830,
  'token_transfers': None,
  'to_ens': None,
  'to_wallet': '0x756F971D1C603f297Ad1020BAf96021a3D9F7AE6',
  'to_iscontract': True,
  'from_ens': None,
  'from_wallet': '0x0d1d649753155e2903e80b89201FFF09E238Eb3B',
  'from_iscontract': False},
 {'value': 0.0,
  'fee': 0.0001226951,
  'tx_types': ['contract_call'],
  'timestamp': '2024-08-28T08:12:58.000000Z',
  'block': 44241,
  'token_transfers': None,
  'to_ens': None,
  'to_wallet': '0x34c124f69e4ABd3D6C88F6f190b8e2f336084779',
 

In [70]:
import pandas as pd
def Ranking(Aw,Bw):
    AllWallets=set()
    for data in Bw:
        if data["color"]=='#000000' or data["color"]=='#27ae60':
            AllWallets.add(data["id"])
    df=pd.DataFrame(Aw)
    A1=pd.pivot_table(df, values="value",index=["to_wallet"],aggfunc="sum").sort_values(by=['value'],ascending=False)
    A2=pd.pivot_table(df, values="value",index=["to_wallet"],aggfunc="count").sort_values(by=['value'],ascending=False)
    B1=pd.pivot_table(df, values="value",index=["from_wallet"],aggfunc="sum").sort_values(by=['value'],ascending=False)
    B2=pd.pivot_table(df, values="value",index=["from_wallet"],aggfunc="count").sort_values(by=['value'],ascending=False)
    Temp=0
    Sal={"To_Sum":[],"To_Count":[],"From_Sum":[],"From_Count":[]}
    for j in [A1,A2,B1,B2]:
        Temp+=1
        Temp3=[]
        for i in range(10):
            Temp2={}
            if j.iloc[i].name in AllWallets:
                Temp2["type"]="Wallet"
            else:
                Temp2["type"]="Contract"
            Temp2["name"]=j.iloc[i].name
            Temp2["value"]=int(j.iloc[i].value)
            Temp3.append(Temp2)
        if Temp==1:
            Sal["To_Sum"]=Temp3
        if Temp==2:
            Sal["To_Count"]=Temp3
        if Temp==3:
            Sal["From_Sum"]=Temp3
        if Temp==4:
            Sal["From_Count"]=Temp3
    return Sal

In [71]:
Sal

{'To_Sum': [{'type': 'Wallet',
   'name': '0xa482AE7d753b33cB06Fd6872443E902cB6BEe592',
   'value': 1600},
  {'type': 'Wallet',
   'name': '0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf',
   'value': 30},
  {'type': 'Wallet',
   'name': '0x0AeA6B0eC79A1060E1736d76e0c38E34459Eb130',
   'value': 4},
  {'type': 'Contract',
   'name': '0x0cddE1982fdB95c93258011636B7d3095D52e9DB',
   'value': 3},
  {'type': 'Contract',
   'name': '0x41B07c0a41B0f3CC9FeCd5FD57fA847E751AF17b',
   'value': 3},
  {'type': 'Wallet',
   'name': '0x763C8EE1F3C022d376771940234812f66ebB30EF',
   'value': 2},
  {'type': 'Contract',
   'name': '0x3B4D31D38b9F4493b7CFC09F61b1110106E5e653',
   'value': 2},
  {'type': 'Contract',
   'name': '0x05412f1f78d7D811623ba49c391D810a20254C92',
   'value': 2},
  {'type': 'Contract',
   'name': '0x0429A2Da7884CA14E53142988D5845952fE4DF6a',
   'value': 2},
  {'type': 'Contract',
   'name': '0x5A4830885f12438E00D8f4d98e9Fe083e707698C',
   'value': 2}],
 'To_Count': [{'type': 'Contract'

In [12]:
AAA=AA['items'][0]

AA['items'][0]

{'timestamp': '2024-08-24T12:53:50.000000Z',
 'fee': {'type': 'actual', 'value': '54769400000000'},
 'gas_limit': '713507',
 'block': 36515,
 'status': 'ok',
 'method': None,
 'confirmations': 2070,
 'type': 2,
 'exchange_rate': None,
 'to': {'ens_domain_name': None,
  'hash': '0x49D8d8a61A7807a8cC78b42A34FA223014518863',
  'implementation_address': None,
  'implementation_name': None,
  'implementations': [],
  'is_contract': False,
  'is_verified': False,
  'metadata': None,
  'name': None,
  'private_tags': [],
  'public_tags': [],
  'watchlist_names': []},
 'tx_burnt_fee': '54769400000000',
 'max_fee_per_gas': '100000000000',
 'result': 'success',
 'hash': '0x4b52d5f444bf6202d823ba73914efbd4ebf9f02f72fa036ad9584ce2ce6da0fe',
 'gas_price': '100000000',
 'priority_fee': '1095388000000000',
 'base_fee_per_gas': '100000000',
 'from': {'ens_domain_name': None,
  'hash': '0xa482AE7d753b33cB06Fd6872443E902cB6BEe592',
  'implementation_address': None,
  'implementation_name': None,
  'impl

In [9]:

import pandas as pd
pd.DataFrame(Aw).columns

Index(['value', 'fee', 'tx_types', 'timestamp', 'block', 'token_transfers',
       'to_ens', 'to_wallet', 'to_iscontract', 'from_ens', 'from_wallet',
       'from_iscontract'],
      dtype='object')

In [59]:
AAA['tx_types']


['coin_transfer']

{'value': '0', 'tx_types': ['contract_creation'], 'timestamp': '2024-08-25T07:53:32.000000Z', 'block': 38078, 'token_transfers': None, 'from_ens': None, 'from_wallet': '0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf', 'from_iscontract': False, 'to_wallet': '0x5E8C933d44b2457D4dA7BC05e0421c2AF00247cA', 'fee': '1493096700000000'}
{'value': '0', 'tx_types': ['contract_creation'], 'timestamp': '2024-08-23T06:56:29.000000Z', 'block': 34529, 'token_transfers': None, 'from_ens': None, 'from_wallet': '0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf', 'from_iscontract': False, 'to_wallet': '0xD39F12a43fD40fdcc4a4881d381Ac9fdbCa3FeAe', 'fee': '1493096700000000'}
{'value': '0', 'tx_types': ['contract_creation'], 'timestamp': '2024-08-22T10:58:50.000000Z', 'block': 33111, 'token_transfers': None, 'from_ens': None, 'from_wallet': '0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf', 'from_iscontract': False, 'to_wallet': '0xc4E9F06c473C5a8B99bA701989fF91AA5C38bf1f', 'fee': '1008031900000000'}
{'value': '0', 'tx_typ

In [ ]:
{'timestamp': '2024-08-25T07:53:32.000000Z', 'fee': {'type': 'actual', 'value': '1493096700000000'}, 'gas_limit': '16662856', 'block': 38078, 'status': 'ok', 'method': None, 'confirmations': 507, 'type': 2, 'exchange_rate': None, 'to': None, 'tx_burnt_fee': '1493096700000000', 'max_fee_per_gas': '100000000', 'result': 'success', 'hash': '0xe75f13946857f06193e300c43efb9606162f0c09250309d6a497308e7ae1dd15', 'gas_price': '100000000', 'priority_fee': '0', 'base_fee_per_gas': '100000000', 'from': {'ens_domain_name': None, 'hash': '0x907fC0C7E6b84F0229c13F57D413F72D33Ff3bAf', 'implementation_address': None, 'implementation_name': None, 'implementations': [], 'is_contract': False, 'is_verified': False, 'metadata': None, 'name': None, 'private_tags': [], 'public_tags': [], 'watchlist_names': []}, 'token_transfers': None, 'tx_types': ['contract_creation'], 'gas_used': '14930967', 'created_contract': {'ens_domain_name': None, 'hash': '0x5E8C933d44b2457D4dA7BC05e0421c2AF00247cA', 'implementation_address': None, 'implementation_name': None, 'implementations': [], 'is_contract': True, 'is_verified': False, 'metadata': None, 'name': None, 'private_tags': [], 'public_tags': [], 'watchlist_names': []}, 'position': 1, 'nonce': 74, 'has_error_in_internal_txs': False, 'actions': [], 'decoded_input': None, 'token_transfers_overflow': None, 'raw_input': '0x6080604052348015600e575f80fd5b50335f806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550611ab38061005b5f395ff3fe608060405260043610610089575f3560e01c8063aca38e6d11610058578063aca38e6d1461014b578063bc89fc7214610175578063c6e134c61461019d578063f851a440146101c5578063fedf1801146101ef57610090565b806313efe858146100945780635cbb3a01146100bc57806366852374146100e4578063a6a4e1ab1461012357610090565b3661009057005b5f80fd5b34801561009f575f80fd5b506100ba60048036038101906100b59190610fa8565b610230565b005b3480156100c7575f80fd5b506100e260048036038101906100dd9190611047565b610413565b005b3480156100ef575f80fd5b5061010a60048036038101906101059190610fa8565b6106b6565b60405161011a949392919061113d565b60405180910390f35b34801561012e575f80fd5b5061014960048036038101906101449190611187565b6107ab565b005b348015610156575f80fd5b5061015f6109e3565b60405161016c919061125a565b60405180910390f35b348015610180575f80fd5b5061019b60048036038101906101969190610fa8565b6109e9565b005b3480156101a8575f80fd5b506101c360048036038101906101be9190611273565b610c28565b005b3480156101d0575f80fd5b506101d9610cfa565b6040516101e6919061129e565b60405180910390f35b3480156101fa575f80fd5b5061021560048036038101906102109190611273565b610d1d565b604051610227969594939291906112b7565b60405180910390f35b8160015f8281526020019081526020015f206004015f9054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff16146102d2576040517f08c379a00000000000000000000000000000000000000000000000000000000081526004016102c99061139b565b60405180910390fd5b60025f8481526020019081526020015f205f8373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020015f2060020160019054906101000a900460ff1661036e576040517f08c379a000000000000000000000000000000000000000000000000000000000815260040161036590611403565b60405180910390fd5b600160025f8581526020019081526020015f205f8473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020015f206002015f6101000a81548160ff0219169083151502179055507fd4829f45099f9fa7e85153a0ea413a85dadd5d09c3ff1baa69160e014c86e4ea8383604051610406929190611421565b60405180910390a1505050565b60015f8481526020019081526020015f2060040160149054906101000a900460ff16610474576040517f08c379a000000000000000000000000000000000000000000000000000000000815260040161046b90611492565b60405180910390fd5b60025f8481526020019081526020015f205f3373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020015f2060020160019054906101000a900460ff1615610511576040517f08c379a000000000000000000000000000000000000000000000000000000000815260040161050890611520565b60405180910390fd5b60405180608001604052803373ffffffffffffffffffffffffffffffffffffffff16815260200183838080601f0160208091040260200160405190810160405280939291908181526020018383808284375f81840152601f19601f8201169050808301925050505050505081526020015f151581526020016001151581525060025f8581526020019081526020015f205f3373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020015f205f820151815f015f6101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555060208201518160010190816106359190611765565b506040820151816002015f6101000a81548160ff02191690831515021790555060608201518160020160016101000a81548160ff0219169083151502179055509050507f394b2f46e841d2b670718e98a15dfb76853abc5a8761449c74e9cba62d90152183336040516106a9929190611421565b60405180910390a1505050565b6002602052815f5260405f20602052805f5260405f205f9150915050805f015f9054906101000a900473ffffffffffffffffffffffffffffffffffffffff169080600101805461070590611598565b80601f016020809104026020016040519081016040528092919081815260200182805461073190611598565b801561077c5780601f106107535761010080835404028352916020019161077c565b820191905f5260205f20905b81548152906001019060200180831161075f57829003601f168201915b505050505090806002015f9054906101000a900460ff16908060020160019054906101000a900460ff16905084565b6040518060c0016040528088888080601f0160208091040260200160405190810160405280939291908181526020018383808284375f81840152601f19601f82011690508083019250505050505050815260200186868080601f0160208091040260200160405190810160405280939291908181526020018383808284375f81840152601f19601f82011690508083019250505050505050815260200184848080601f0160208091040260200160405190810160405280939291908181526020018383808284375f81840152601f19601f8201169050808301925050505050505081526020018281526020013373ffffffffffffffffffffffffffffffffffffffff1681526020016001151581525060015f60035481526020019081526020015f205f820151815f0190816108e09190611765565b5060208201518160010190816108f69190611765565b50604082015181600201908161090c9190611765565b50606082015181600301556080820151816004015f6101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555060a08201518160040160146101000a81548160ff0219169083151502179055509050507fb95f0f805f90a93611b82f20f5f8abf84ef8bafb9146dc64f41bada29a378dad60035488888888336040516109bb9695949392919061186e565b60405180910390a160035f8154809291906109d5906118f0565b919050555050505050505050565b60035481565b8160015f8281526020019081526020015f206004015f9054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff1614610a8b576040517f08c379a0000000000000000000000000000000000000000000000000000000008152600401610a829061139b565b60405180910390fd5b60025f8481526020019081526020015f205f8373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020015f206002015f9054906101000a900460ff16610b26576040517f08c379a0000000000000000000000000000000000000000000000000000000008152600401610b1d90611981565b60405180910390fd5b5f60015f8581526020019081526020015f206003015490505f8373ffffffffffffffffffffffffffffffffffffffff1682604051610b63906119cc565b5f6040518083038185875af1925050503d805f8114610b9d576040519150601f19603f3d011682016040523d82523d5f602084013e610ba2565b606091505b5050905080610be6576040517f08c379a0000000000000000000000000000000000000000000000000000000008152600401610bdd90611a2a565b60405180910390fd5b7f8a9c00e0330ad6bdd028b5f904d12736ca2c437d2321f25af05bd5c51cc80c9b858584604051610c1993929190611a48565b60405180910390a15050505050565b8060015f8281526020019081526020015f206004015f9054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff1614610cca576040517f08c379a0000000000000000000000000000000000000000000000000000000008152600401610cc19061139b565b60405180910390fd5b5f60015f8481526020019081526020015f2060040160146101000a81548160ff0219169083151502179055505050565b5f8054906101000a900473ffffffffffffffffffffffffffffffffffffffff1681565b6001602052805f5260405f205f91509050805f018054610d3c90611598565b80601f0160208091040260200160405190810160405280929190818152602001828054610d6890611598565b8015610db35780601f10610d8a57610100808354040283529160200191610db3565b820191905f5260205f20905b815481529060010190602001808311610d9657829003601f168201915b505050505090806001018054610dc890611598565b80601f0160208091040260200160405190810160405280929190818152602001828054610df490611598565b8015610e3f5780601f10610e1657610100808354040283529160200191610e3f565b820191905f5260205f20905b815481529060010190602001808311610e2257829003601f168201915b505050505090806002018054610e5490611598565b80601f0160208091040260200160405190810160405280929190818152602001828054610e8090611598565b8015610ecb5780601f10610ea257610100808354040283529160200191610ecb565b820191905f5260205f20905b815481529060010190602001808311610eae57829003601f168201915b505050505090806003015490806004015f9054906101000a900473ffffffffffffffffffffffffffffffffffffffff16908060040160149054906101000a900460ff16905086565b5f80fd5b5f80fd5b5f819050919050565b610f2d81610f1b565b8114610f37575f80fd5b50565b5f81359050610f4881610f24565b92915050565b5f73ffffffffffffffffffffffffffffffffffffffff82169050919050565b5f610f7782610f4e565b9050919050565b610f8781610f6d565b8114610f91575f80fd5b50565b5f81359050610fa281610f7e565b92915050565b5f8060408385031215610fbe57610fbd610f13565b5b5f610fcb85828601610f3a565b9250506020610fdc85828601610f94565b9150509250929050565b5f80fd5b5f80fd5b5f80fd5b5f8083601f84011261100757611006610fe6565b5b8235905067ffffffffffffffff81111561102457611023610fea565b5b6020830191508360018202830111156110405761103f610fee565b5b9250929050565b5f805f6040848603121561105e5761105d610f13565b5b5f61106b86828701610f3a565b935050602084013567ffffffffffffffff81111561108c5761108b610f17565b5b61109886828701610ff2565b92509250509250925092565b6110ad81610f6d565b82525050565b5f81519050919050565b5f82825260208201905092915050565b8281835e5f83830152505050565b5f601f19601f8301169050919050565b5f6110f5826110b3565b6110ff81856110bd565b935061110f8185602086016110cd565b611118816110db565b840191505092915050565b5f8115159050919050565b61113781611123565b82525050565b5f6080820190506111505f8301876110a4565b818103602083015261116281866110eb565b9050611171604083018561112e565b61117e606083018461112e565b95945050505050565b5f805f805f805f6080888a0312156111a2576111a1610f13565b5b5f88013567ffffffffffffffff8111156111bf576111be610f17565b5b6111cb8a828b01610ff2565b9750975050602088013567ffffffffffffffff8111156111ee576111ed610f17565b5b6111fa8a828b01610ff2565b9550955050604088013567ffffffffffffffff81111561121d5761121c610f17565b5b6112298a828b01610ff2565b9350935050606061123c8a828b01610f3a565b91505092959891949750929550565b61125481610f1b565b82525050565b5f60208201905061126d5f83018461124b565b92915050565b5f6020828403121561128857611287610f13565b5b5f61129584828501610f3a565b91505092915050565b5f6020820190506112b15f8301846110a4565b92915050565b5f60c0820190508181035f8301526112cf81896110eb565b905081810360208301526112e381886110eb565b905081810360408301526112f781876110eb565b9050611306606083018661124b565b61131360808301856110a4565b61132060a083018461112e565b979650505050505050565b7f4f6e6c792074686520656d706c6f7965722063616e20657865637574652074685f8201527f6973000000000000000000000000000000000000000000000000000000000000602082015250565b5f6113856022836110bd565b91506113908261132b565b604082019050919050565b5f6020820190508181035f8301526113b281611379565b9050919050565b7f4170706c69636174696f6e20646f6573206e6f742065786973740000000000005f82015250565b5f6113ed601a836110bd565b91506113f8826113b9565b602082019050919050565b5f6020820190508181035f83015261141a816113e1565b9050919050565b5f6040820190506114345f83018561124b565b61144160208301846110a4565b9392505050565b7f496e7465726e73686970206973206e6f206c6f6e6765722061637469766500005f82015250565b5f61147c601e836110bd565b915061148782611448565b602082019050919050565b5f6020820190508181035f8301526114a981611470565b9050919050565b7f596f75206861766520616c7265616479206170706c69656420666f72207468695f8201527f7320696e7465726e736869700000000000000000000000000000000000000000602082015250565b5f61150a602c836110bd565b9150611515826114b0565b604082019050919050565b5f6020820190508181035f830152611537816114fe565b9050919050565b7f4e487b71000000000000000000000000000000000000000000000000000000005f52604160045260245ffd5b7f4e487b71000000000000000000000000000000000000000000000000000000005f52602260045260245ffd5b5f60028204905060018216806115af57607f821691505b6020821081036115c2576115c161156b565b5b50919050565b5f819050815f5260205f209050919050565b5f6020601f8301049050919050565b5f82821b905092915050565b5f600883026116247fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff826115e9565b61162e86836115e9565b95508019841693508086168417925050509392505050565b5f819050919050565b5f61166961166461165f84610f1b565b611646565b610f1b565b9050919050565b5f819050919050565b6116828361164f565b61169661168e82611670565b8484546115f5565b825550505050565b5f90565b6116aa61169e565b6116b5818484611679565b505050565b5b818110156116d8576116cd5f826116a2565b6001810190506116bb565b5050565b601f82111561171d576116ee816115c8565b6116f7846115da565b81016020851015611706578190505b61171a611712856115da565b8301826116ba565b50505b505050565b5f82821c905092915050565b5f61173d5f1984600802611722565b1980831691505092915050565b5f611755838361172e565b9150826002028217905092915050565b61176e826110b3565b67ffffffffffffffff8111156117875761178661153e565b5b6117918254611598565b61179c8282856116dc565b5f60209050601f8311600181146117cd575f84156117bb578287015190505b6117c5858261174a565b86555061182c565b601f1984166117db866115c8565b5f5b82811015611802578489015182556001820191506020850194506020810190506117dd565b8683101561181f578489015161181b601f89168261172e565b8355505b6001600288020188555050505b505050505050565b828183375f83830152505050565b5f61184d83856110bd565b935061185a838584611834565b611863836110db565b840190509392505050565b5f6080820190506118815f83018961124b565b8181036020830152611894818789611842565b905081810360408301526118a9818587611842565b90506118b860608301846110a4565b979650505050505050565b7f4e487b71000000000000000000000000000000000000000000000000000000005f52601160045260245ffd5b5f6118fa82610f1b565b91507fffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff820361192c5761192b6118c3565b5b600182019050919050565b7f4170706c69636174696f6e206e6f7420617070726f76656400000000000000005f82015250565b5f61196b6018836110bd565b915061197682611937565b602082019050919050565b5f6020820190508181035f8301526119988161195f565b9050919050565b5f81905092915050565b50565b5f6119b75f8361199f565b91506119c2826119a9565b5f82019050919050565b5f6119d6826119ac565b9150819050919050565b7f4661696c656420746f2073656e642073746970656e64000000000000000000005f82015250565b5f611a146016836110bd565b9150611a1f826119e0565b602082019050919050565b5f6020820190508181035f830152611a4181611a08565b9050919050565b5f606082019050611a5b5f83018661124b565b611a6860208301856110a4565b611a75604083018461124b565b94935050505056fea264697066735822122004b5b9f4be70347e97d161dc99e9ec1c8422678627e48e08a4e035709267b30764736f6c634300081a0033', 'value': '0', 'max_priority_fee_per_gas': '100000000', 'revert_reason': None, 'confirmation_duration': [0, 28104.0], 'tx_tag': None}


In [ ]:
#Blocks

In [76]:
import requests
import json
import pandas as pd
def metricsBlocks():
    response=json.loads(requests.get("https://opencampus-codex.blockscout.com/api/v2/blocks?type=block").content)
    return BlocksIndex(response)

def BlocksIndex(items):
    list_data=[]
    for item in items["items"]:
        data=  {"hash":item['hash'],
        'height':item['height'],
        'size':item['size'],
        'miner':item['miner']['hash'],
        'difficulty':item['difficulty'],
        'burnt_fees':item['burnt_fees'],
        'base_fee_per_gas':item['base_fee_per_gas'],
        'parent_hash':item['parent_hash'],
        'total_difficulty':item['total_difficulty'],
            'tx_count':item['tx_count'],
            'timestamp':item['timestamp'],
            "gas_used":int(item['gas_used']),
        'gas_limit':item['gas_limit'],
        'tx_fees':int(item['tx_fees'])
        }
    list_data.append(data)
    df=pd.DataFrame(list_data)
    mean_size=int(df["size"].mean())
    mean_tx_count=int(df['tx_count'].mean())
    mean_tx_fees=int(df['tx_fees'].mean())
    mean_gas_used=int(df["gas_used"].mean())
    output={"items":list_data,"mean_size":mean_size,"mean_tx_count":mean_tx_count,"mean_tx_fees":mean_tx_fees,"mean_gas_used":mean_gas_used}
    return output


##Index
# 'height' "hash" 'tx_count'


#head hash, 'height', 'tx_count', 'timestamp'
#block 'height','size','parent_hash','miner'
#gas, fees and others "gas_used" 'gas_limit' 'burnt_fees' 'base_fee_per_gas' 'total_difficulty' 'difficulty'

def stadistics_index(items):
    output={'network_utilization_percentage':items['network_utilization_percentage'],
            'total_blocks':items['total_blocks'],
            'total_transactions':items['total_transactions'],
            'transactions_today':items['transactions_today'],
            'total_addresses':items['total_addresses'],
            'gas_used_today':items['gas_used_today'],
            'average_block_time':int(items['average_block_time'])
    }
    return output
def stadistics_blockchain():
    response=json.loads(requests.get("https://opencampus-codex.blockscout.com/api/v2/stats").content)
    return stadistics_index(response)
def metricsTx():
    response=json.loads(requests.get("https://opencampus-codex.blockscout.com/api/v2/transactions?filter=validated").content)
    return TxIndex(response)
A=metricsTx()


def TxIndex(items):
    list_data=[]
    for item in items["items"]:
        data=  {'timestamp':item['timestamp'],
        "fee":int(item["fee"]["value"]),
        'block':item["block"],
        'method':item["method"],
        "to":item["to"]["hash"],
        "from":item["from"]["hash"],
        'tx_burnt_fee':item['tx_burnt_fee'],
        "hash":item["hash"],
        'priority_fee':item['priority_fee'],
        'tx_types':item['tx_types'],
        'gas_used':int(item['gas_used']),
        'created_contract':item['created_contract'],
        }
        list_data.append(data)
    df=pd.DataFrame(list_data)
    mean_fee=int(df["fee"].mean())
    mean_gas_used=int(df["gas_used"].mean())
    output={"items":list_data,"mean_fee":mean_fee,"mean_gas_used":mean_gas_used}
    return output 


BlocksToIndex=metricsBlocks()
Data_general=stadistics_blockchain()
TxtoIndex=metricsTx()


In [3]:
import requests
import json
import pandas as pd
def TxDetail(hash):
    response=json.loads(requests.get("https://opencampus-codex.blockscout.com/api/v2/transactions/%s"%(hash)).content)
    item=response
    data=  {'timestamp':item['timestamp'],
        "fee":int(item["fee"]["value"]),
        'block':item["block"],
        'method':item["method"],
        "to":item["to"]["hash"],
        "from":item["from"]["hash"],
        'tx_burnt_fee':item['tx_burnt_fee'],
        "hash":item["hash"],
        'priority_fee':item['priority_fee'],
        'tx_types':item['tx_types'],
        'gas_used':int(item['gas_used']),
        'created_contract':item['created_contract'],
        }
    return data
hash="0xc70d49d8426bf37f24d1913d1c2652e16e739cf5ff2f4db154a770622e01ef67"
TxDetail(hash)

{'timestamp': '2024-09-08T11:31:15.000000Z',
 'fee': 104202400000000,
 'block': 55880,
 'method': '0x3161b7f6',
 'to': '0x8b14D287B4150Ff22Ac73DF8BE720e933f659abc',
 'from': '0xF5E8A439C599205C1aB06b535DE46681Aed1007a',
 'tx_burnt_fee': None,
 'hash': '0xc70d49d8426bf37f24d1913d1c2652e16e739cf5ff2f4db154a770622e01ef67',
 'priority_fee': None,
 'tx_types': ['contract_call'],
 'gas_used': 1042024,
 'created_contract': None}

In [4]:
def BlockDetail(block):
    response=json.loads(requests.get("https://opencampus-codex.blockscout.com/api/v2/blocks/%s"%(block)).content)
    item=response
    data=  {"hash":item['hash'],
        'height':item['height'],
        'size':item['size'],
        'miner':item['miner']['hash'],
        'difficulty':item['difficulty'],
        'burnt_fees':item['burnt_fees'],
        'base_fee_per_gas':item['base_fee_per_gas'],
        'parent_hash':item['parent_hash'],
        'total_difficulty':item['total_difficulty'],
            'tx_count':item['tx_count'],
            'timestamp':item['timestamp'],
            "gas_used":int(item['gas_used']),
        'gas_limit':item['gas_limit'],
        'tx_fees':int(item['tx_fees'])
        }  
    return data


In [19]:
import private.modelcontract as LLMC
def isContract(hash):
    A=requests.get('https://opencampus-codex.blockscout.com/api/v2/smart-contracts/%s'%(hash))
    if A.status_code==200:
        return True
    else:
        return False
def contractInfo(item):
    data={"is_vyper_contract":item['is_vyper_contract'],
            'is_fully_verified':item['is_fully_verified'],
            'is_blueprint':item['is_blueprint'],
            'source_code':item['source_code'],
            'verified_at': item['verified_at'],
            'is_verified': item['is_verified'],
            'name': item['name'],
            'language': item['language']
    }
    return data
def contractsDetail(hash):
    if not isContract(hash):
        return {"message":"No contract"}
    response=json.loads(requests.get("https://opencampus-codex.blockscout.com/api/v2/smart-contracts/%s"%(hash)).content)
    data=contractInfo(response)
    DetailContract=LLMC.AnalysisContract(data["source_code"])
    try:
        DetailContract=json.loads(DetailContract)
    except:
        DetailContract=json.loads(DetailContract[DetailContract.find("{"):DetailContract.find("}")+1])
    return data,DetailContract



contractsDetail("0xb7F47AAa68959aa69D2dF00dfa92FdD024a47889")


({'is_vyper_contract': False,
  'is_fully_verified': True,
  'is_blueprint': False,
  'source_code': '// SPDX-License-Identifier: MIT\npragma solidity ^0.8.0;\n\nimport "@openzeppelin/contracts/access/Ownable.sol";\nimport "@openzeppelin/contracts/access/AccessControl.sol";\nimport "@openzeppelin/contracts/utils/ReentrancyGuard.sol";\n\ncontract ScholarshipPlatform is Ownable, AccessControl, ReentrancyGuard {\n    bytes32 public constant ORGANIZATION_ROLE = keccak256("ORGANIZATION_ROLE");\n\n    struct UserProfile {\n        string name;\n        string role; // Organization, Student\n        string email; // Email of the user\n        bool verified; // Verification status for organizations\n        uint256[] createdScholarships; // List of scholarships created by the organization\n        uint256[] receivedScholarships; // List of scholarships received by the student\n        uint256 totalReceived; // Total amount received by the student\n        uint256[] appliedScholarships; // List

In [6]:
A=contractsDetail("0xb7F47AAa68959aa69D2dF00dfa92FdD024a47889")
B=contractsDetail("0x534e5c6d0aB4Ad1837e9338bF3bdf05f988fEfc3")




In [7]:
B["source_code"]

'// SPDX-License-Identifier: MIT\npragma solidity 0.8.9;\n\nimport "@openzeppelin/contracts/token/ERC20/ERC20.sol";\n\n/// @title WEDU (Wrapped EduChain) Token\n/// @notice This contract wraps EduChain (EDU) native tokens into ERC20-compliant WEDU tokens.\ncontract WEDU is ERC20 {\n    \n    /// @notice Constructor initializes the ERC20 token with name and symbol.\n    constructor() ERC20("Wrapped EduChain", "WEDU") {}\n\n    /// @notice Fallback function to receive EDU and automatically wrap it into WEDU.\n    /// The received EDU will be wrapped as WEDU and credited to the sender.\n    receive() external payable {\n        deposit();\n    }\n\n    /// @notice Wraps EDU into WEDU tokens. The amount of WEDU minted equals the amount of EDU sent.\n    /// @dev This function mints WEDU tokens equivalent to the amount of EDU sent by the user.\n    function deposit() public payable {\n        _mint(msg.sender, msg.value);\n    }\n\n    /// @notice Unwraps WEDU tokens back into EDU.\n    ///

In [19]:
import os
def extractConfig(nameModel="SystemData",relPath=os.path.join("private/experiment_config.json"),dataOut="keygroq"):
    configPath=os.path.join(os.getcwd(),relPath)
    with open(configPath, 'r', encoding='utf-8') as file:
        config = json.load(file)[nameModel]
    Output= config[dataOut]
    return Output
keygroq=extractConfig(nameModel="SystemData",dataOut="keygroq")
keygroq

'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'

In [ ]:
You are an expert in blockchain smart contract analysis. You will be provided with the source code of a smart contract. Your task is to perform a comprehensive analysis of the contract, covering the following aspects:

Functionality Analysis:

- Identify and explain the main functions of the contract.
- Describe the general purpose of the contract.
- Analyze how different parts of the code interact.

Sector and Use Classification:

Determine in which sector(s) of the blockchain industry this contract could be used. The sectors you can mention are:
- DeFi
- NFTs
- Governance
- Play-to-Earn Games
- Prediction Markets
- Reward Systems in Content Platforms
- Fungible Tokens
- Real Asset Tokenization Systems
- Self-Sovereign Identity Systems
- Credential Verification
- Decentralized Reputation Management

Identification of Potential Issues:

- Review the code for common security vulnerabilities in smart contracts.
- Identify possible inefficiencies in gas usage.
- Point out areas where the code could be improved in terms of clarity or optimization.
- Warn about any practices that could lead to scalability or maintenance issues.

Summary and Recommendations:

- Provide a concise summary of key findings.
- Offer recommendations for improving the contract.

Format a json:

{"Functionality Analysis":[Your detailed analysis here], "Sector and Use Classification":[Your classification and without explanation here],"Identification of Potential Issues":[Your list of potential problems here],"Summary and Recommendations":[Your summary and recommendations here]}


Ensure that your analysis is detailed yet concise, using appropriate technical terminology for smart contracts and blockchain development. If you find anything unusual or innovative in the code, make sure to mention it.

Smart Contract Code to analyze:

'// SPDX-License-Identifier: MIT\npragma solidity ^0.8.0;\n\nimport "@openzeppelin/contracts/access/Ownable.sol";\nimport "@openzeppelin/contracts/access/AccessControl.sol";\nimport "@openzeppelin/contracts/utils/ReentrancyGuard.sol";\n\ncontract ScholarshipPlatform is Ownable, AccessControl, ReentrancyGuard {\n    bytes32 public constant ORGANIZATION_ROLE = keccak256("ORGANIZATION_ROLE");\n\n    struct UserProfile {\n        string name;\n        string role; // Organization, Student\n        string email; // Email of the user\n        bool verified; // Verification status for organizations\n        uint256[] createdScholarships; // List of scholarships created by the organization\n        uint256[] receivedScholarships; // List of scholarships received by the student\n        uint256 totalReceived; // Total amount received by the student\n        uint256[] appliedScholarships; // List of scholarships the student applied for\n    }\n\n    struct ScholarshipInfo {\n        uint256 amount;\n        string description;\n        ScholarshipCategory category;\n        uint256 startDate;\n        uint256 endDate;\n        uint256 maxApprovalCount;\n        uint256 currentApprovalCount;\n        bool active;\n        address listedBy;\n        string[] applicationRequirements; // List of additional requirements for applying\n        mapping(address => string) feedback; // Mapping of student addresses to their feedback\n        uint8 ratingSum; // Sum of all ratings received\n        uint8 ratingCount; // Number of ratings received\n    }\n\n    struct Application {\n        uint256 scholarshipId;\n        address applicant;\n        string reason;\n        bool approved;\n        bool rejected;\n        uint256 appliedAt;\n    }\n\n    enum ScholarshipCategory { College, School, Project, Work, Research }\n\n    mapping(address => UserProfile) public profiles;\n    ScholarshipInfo[] public scholarships;\n    mapping(uint256 => Application) public applications;\n    mapping(uint256 => uint256[]) public scholarshipApplications; // Maps scholarshipId to list of application IDs\n    uint256 public applicationCount;\n\n    event UserRegistered(address indexed user, string name, string role);\n    event RoleAssigned(address indexed user, string role);\n    event RoleRemoved(address indexed user, string role);\n    event ScholarshipAdded(uint256 indexed scholarshipId, string description, uint256 amount);\n    event ScholarshipUpdated(uint256 indexed scholarshipId, string description, uint256 amount);\n    event ScholarshipDeactivated(uint256 indexed scholarshipId);\n    event ApplicationSubmitted(uint256 indexed applicationId, address indexed applicant, uint256 scholarshipId);\n    event ApplicationApproved(uint256 indexed applicationId, uint256 amount);\n    event ApplicationRejected(uint256 indexed applicationId);\n    event FundsTransferred(address indexed recipient, uint256 amount);\n    event ScholarshipFeedbackSubmitted(uint256 indexed scholarshipId, address indexed student, string feedback, uint8 rating);\n\n    constructor(address admin) Ownable(admin) {\n        _grantRole(DEFAULT_ADMIN_ROLE, admin);\n    }\n\n    // Register a user\n    function registerUser(string calldata name, string calldata role, string calldata email) external nonReentrant {\n    require(bytes(profiles[msg.sender].name).length == 0, "User already registered");\n\n    // Initialize the UserProfile struct without using \'new\' for dynamic arrays\n    UserProfile storage userProfile = profiles[msg.sender];\n    userProfile.name = name;\n    userProfile.role = role;\n    userProfile.email = email;\n    userProfile.verified = false;\n    userProfile.totalReceived = 0;\n\n    emit UserRegistered(msg.sender, name, role);\n    }\n\n    // Verify an organization (Admin verifies an organization and assigns the role)\n    function verifyOrganization(address organization) external onlyRole(DEFAULT_ADMIN_ROLE) {\n        require(bytes(profiles[organization].name).length != 0, "Organization not registered");\n        profiles[organization].verified = true;\n        grantRole(ORGANIZATION_ROLE, organization);\n    }\n\n    // Register scholarship by verified organization\n    function addScholarship(\n        uint256 amount,\n        string memory description,\n        ScholarshipCategory category,\n        uint256 startDate,\n        uint256 endDate,\n        uint256 maxApprovalCount,\n        string[] memory requirements\n    ) public onlyRole(ORGANIZATION_ROLE) payable {\n        require(profiles[msg.sender].verified, "Organization not verified");\n        require(amount > 0, "Amount should be greater than 0");\n        require(msg.value == amount * maxApprovalCount, "Incorrect total fund amount sent");\n\n        scholarships.push();\n        ScholarshipInfo storage scholarship = scholarships[scholarships.length - 1];\n        scholarship.amount = amount;\n        scholarship.description = description;\n        scholarship.category = category;\n        scholarship.startDate = startDate;\n        scholarship.endDate = endDate;\n        scholarship.maxApprovalCount = maxApprovalCount;\n        scholarship.currentApprovalCount = 0;\n        scholarship.active = true;\n        scholarship.listedBy = msg.sender;\n        scholarship.applicationRequirements = requirements;\n\n        profiles[msg.sender].createdScholarships.push(scholarships.length - 1);\n\n        emit ScholarshipAdded(scholarships.length - 1, description, amount);\n    }\n\n    // Update scholarship details\n    function updateScholarship(\n        uint256 scholarshipId,\n        string memory description,\n        uint256 endDate,\n        uint256 maxApprovalCount,\n        string[] memory requirements\n    ) public onlyRole(ORGANIZATION_ROLE) {\n        ScholarshipInfo storage scholarship = scholarships[scholarshipId];\n        require(scholarship.active, "Scholarship is inactive");\n        require(scholarship.listedBy == msg.sender, "Not the lister");\n\n        scholarship.description = description;\n        scholarship.endDate = endDate;\n        scholarship.maxApprovalCount = maxApprovalCount;\n        scholarship.applicationRequirements = requirements;\n\n        emit ScholarshipUpdated(scholarshipId, description, scholarship.amount);\n    }\n\n    // Deactivate scholarship\n    function deactivateScholarship(uint256 scholarshipId) public onlyRole(ORGANIZATION_ROLE) {\n        ScholarshipInfo storage scholarship = scholarships[scholarshipId];\n        require(scholarship.listedBy == msg.sender, "Not the lister");\n        scholarship.active = false;\n\n        emit ScholarshipDeactivated(scholarshipId);\n    }\n\n    // Apply for scholarship\n    function applyForScholarship(uint256 scholarshipId, string memory reason) public nonReentrant {\n        ScholarshipInfo storage scholarship = scholarships[scholarshipId];\n        require(scholarship.active, "Scholarship is inactive");\n        require(block.timestamp < scholarship.endDate, "Scholarship application period has ended");\n\n        // Check if the student has already applied for this scholarship\n        for (uint256 i = 0; i < profiles[msg.sender].appliedScholarships.length; i++) {\n            require(profiles[msg.sender].appliedScholarships[i] != scholarshipId, "Already applied for this scholarship");\n        }\n\n        applicationCount++;\n        applications[applicationCount] = Application({\n            scholarshipId: scholarshipId,\n            applicant: msg.sender,\n            reason: reason,\n            approved: false,\n            rejected: false,\n            appliedAt: block.timestamp\n        });\n\n        profiles[msg.sender].appliedScholarships.push(scholarshipId);\n        scholarshipApplications[scholarshipId].push(applicationCount);\n\n        emit ApplicationSubmitted(applicationCount, msg.sender, scholarshipId);\n    }\n\n    // Approve application and transfer funds to student\n    function approveApplication(uint256 applicationId) public onlyRole(ORGANIZATION_ROLE) nonReentrant {\n        Application storage application = applications[applicationId];\n        require(!application.approved, "Application already approved");\n        require(!application.rejected, "Application already rejected");\n\n        ScholarshipInfo storage scholarship = scholarships[application.scholarshipId];\n        require(scholarship.currentApprovalCount < scholarship.maxApprovalCount, "Max approval limit reached");\n\n        scholarship.currentApprovalCount++;\n        application.approved = true;\n\n        // Transfer the funds to the student\n        (bool success, ) = application.applicant.call{value: scholarship.amount}("");\n        require(success, "Transfer to student failed");\n\n        // Record the scholarship received by the student\n        profiles[application.applicant].receivedScholarships.push(application.scholarshipId);\n        profiles[application.applicant].totalReceived += scholarship.amount;\n\n        emit ApplicationApproved(applicationId, scholarship.amount);\n        emit FundsTransferred(application.applicant, scholarship.amount);\n    }\n\n    // Reject application\n    function rejectApplication(uint256 applicationId) public onlyRole(ORGANIZATION_ROLE) nonReentrant {\n        Application storage application = applications[applicationId];\n        require(!application.approved, "Application already approved");\n        require(!application.rejected, "Application already rejected");\n\n        application.rejected = true;\n        emit ApplicationRejected(applicationId);\n    }\n\n    // Withdraw unused funds after scholarship ends\n    function withdrawUnusedFunds(uint256 scholarshipId) public onlyRole(ORGANIZATION_ROLE) nonReentrant {\n        ScholarshipInfo storage scholarship = scholarships[scholarshipId];\n        require(block.timestamp > scholarship.endDate, "Scholarship period has not ended");\n        require(scholarship.listedBy == msg.sender, "Not the lister");\n        require(scholarship.active, "Scholarship is inactive");\n\n        uint256 remainingFunds = scholarship.amount * (scholarship.maxApprovalCount - scholarship.currentApprovalCount);\n        scholarship.active = false;\n\n        (bool success, ) = msg.sender.call{value: remainingFunds}("");\n        require(success, "Withdrawal failed");\n    }\n\n    // Submit feedback and rating for a scholarship\n    function submitFeedback(uint256 scholarshipId, string memory feedback, uint8 rating) public nonReentrant {\n        require(rating > 0 && rating <= 5, "Invalid rating"); // Ratings must be between 1 and 5\n        ScholarshipInfo storage scholarship = scholarships[scholarshipId];\n        require(isStudent(msg.sender), "Only students can submit feedback");\n\n        scholarship.feedback[msg.sender] = feedback;\n        scholarship.ratingSum += rating;\n        scholarship.ratingCount += 1;\n\n        emit ScholarshipFeedbackSubmitted(scholarshipId, msg.sender, feedback, rating);\n    }\n\n    // Get an organization\'s created scholarships\n    function getOrganizationScholarships(address organization) external view returns (uint256[] memory) {\n        require(hasRole(ORGANIZATION_ROLE, organization), "Not an organization");\n        return profiles[organization].createdScholarships;\n    }\n\n    // Get a student\'s received scholarships and total amount received\n    function getStudentScholarships(address student) external view returns (uint256[] memory, uint256) {\n        require(keccak256(abi.encodePacked(profiles[student].role)) == keccak256("Student"), "Not a student");\n        return (profiles[student].receivedScholarships, profiles[student].totalReceived);\n    }\n\n    // Get a student\'s feedback on a scholarship\n    function getScholarshipFeedback(uint256 scholarshipId, address student) external view returns (string memory) {\n        require(keccak256(abi.encodePacked(profiles[student].role)) == keccak256("Student"), "Not a student");\n        ScholarshipInfo storage scholarship = scholarships[scholarshipId];\n        return scholarship.feedback[student];\n    }\n\n    // Get the average rating of a scholarship\n    function getScholarshipRating(uint256 scholarshipId) external view returns (uint8) {\n        ScholarshipInfo storage scholarship = scholarships[scholarshipId];\n        if (scholarship.ratingCount == 0) {\n            return 0; // No ratings yet\n        }\n        return scholarship.ratingSum / scholarship.ratingCount;\n    }\n\n    // Get all applications for a specific scholarship\n    function getApplicationsForScholarship(uint256 scholarshipId) external view returns (Application[] memory) {\n        uint256[] memory applicationIds = scholarshipApplications[scholarshipId];\n        Application[] memory scholarshipApplicationsList = new Application[](applicationIds.length);\n\n        for (uint256 i = 0; i < applicationIds.length; i++) {\n            scholarshipApplicationsList[i] = applications[applicationIds[i]];\n        }\n\n        return scholarshipApplicationsList;\n    }\n\n    // Fallback function to receive funds\n    receive() external payable {}\n\n    // Helper function to check if an address is a student\n    function isStudent(address user) internal view returns (bool) {\n        return keccak256(abi.encodePacked(profiles[user].role)) == keccak256("Student");\n    }\n}\n'

Please proceed with your analysis based on the provided code.

In [15]:

import os
q1="""Eres un experto en análisis de contratos inteligentes en blockchain. Se te proporcionará el código fuente de un smart contract. Tu tarea es realizar un análisis exhaustivo del contrato, cubriendo los siguientes aspectos:

Análisis de Funcionalidades:

Identifica y explica las principales funciones del contrato.
Describe el propósito general del contrato.
Analiza cómo interactúan las diferentes partes del código.


Clasificación de Sectores y Usos:

Determina en qué sector(s) de la industria blockchain se podría utilizar este contrato. Los sectores que puedes mencionar son DeFi, NFTs, gobernanza, jJuegos Play-to-Earn, Mercados de predicción, Sistemas de recompensas en plataformas de contenido,Tokens fungibles,Sistemas de tokenización de activos reales,Sistemas de identidad auto-soberana,Verificación de credenciales,Gestión de reputación descentralizada


Identificación de Posibles Problemas:

Revisa el código en busca de vulnerabilidades de seguridad comunes en smart contracts.
Identifica posibles ineficiencias en el uso de gas.
Señala áreas donde el código podría mejorarse en términos de claridad o optimización.
Advierte sobre cualquier práctica que pudiera llevar a problemas de escalabilidad o mantenimiento.


Resumen y Recomendaciones:

Proporciona un resumen conciso de los hallazgos clave.
Ofrece recomendaciones para mejorar el contrato.




Formato de tu respuesta:

Análisis de Funcionalidades
[Tu análisis detallado aquí]
Clasificación de Sectores y Usos
[Tu clasificación y explicación aquí]
Identificación de Posibles Problemas
[Tu lista de problemas potenciales aquí]
Resumen y Recomendaciones
[Tu resumen y recomendaciones aquí]

Asegúrate de que tu análisis sea detallado pero conciso, utilizando terminología técnica apropiada para smart contracts y desarrollo blockchain. Si encuentras algo inusual o innovador en el código, asegúrate de mencionarlo.
Código del Smart Contract a analizar:

"""


B="""You are an expert in blockchain smart contract analysis. You will be provided with the source code of a smart contract. Your task is to perform a comprehensive analysis of the contract, covering the following aspects:

Functionality Analysis:

- Identify and explain the main functions of the contract.
- Describe the general purpose of the contract.
- Analyze how different parts of the code interact.

Sector and Use Classification:

Determine in which sector(s) of the blockchain industry this contract could be used. The sectors you can mention are:
- DeFi
- NFTs
- Governance
- Play-to-Earn Games
- Prediction Markets
- Reward Systems in Content Platforms
- Fungible Tokens
- Real Asset Tokenization Systems
- Self-Sovereign Identity Systems
- Credential Verification
- Decentralized Reputation Management

Identification of Potential Issues:

- Review the code for common security vulnerabilities in smart contracts.
- Identify possible inefficiencies in gas usage.
- Point out areas where the code could be improved in terms of clarity or optimization.
- Warn about any practices that could lead to scalability or maintenance issues.

Summary and Recommendations:

- Provide a concise summary of key findings.
- Offer recommendations for improving the contract.

Format a json:

{"Functionality Analysis":[Your detailed analysis here], "Sector and Use Classification":[Your classification and without explanation here],"Identification of Potential Issues":[Your list of potential problems here],"Summary and Recommendations":[Your summary and recommendations here]}

Ensure that your analysis is detailed yet concise, using appropriate technical terminology for smart contracts and blockchain development. If you find anything unusual or innovative in the code, make sure to mention it.

Smart Contract Code to analyze:

%s

Please proceed with your analysis based on the provided code.

"""
def AnalysisContract(source):
    from groq import Groq
    client = Groq(
    api_key="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",
    )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": "Explain the importance of fast language models",
            }
        ],
        model="llama-3.1-70b-versatile",
    )
    print(chat_completion.choices[0].message.content)
AnalysisContract(A["source_code"])

Fast language models have numerous benefits, transforming the way we interact with technology and improving various aspects of our lives. Here are some key importance of fast language models:

1. **Efficient Processing**: Fast language models can process and generate text quickly, reducing latency and enabling real-time applications such as chatbots, voice assistants, and language translation systems.
2. **Improved User Experience**: Faster language models enhance user experience by providing rapid responses, making interactions more engaging, and reducing wait times.
3. **Increased Productivity**: Fast language models can help automate tasks such as text summarization, content generation, and language translation, freeing up time for more strategic and creative work.
4. **Enhanced Accessibility**: Fast language models can facilitate communication for people with disabilities, such as those who are deaf or hard of hearing, by providing real-time text-to-speech and speech-to-text system

In [8]:
import private.modelcontract as ter
SD=ter.AnalysisContract(A["source_code"])

In [31]:
import os
import json
def extractConfig(nameModel="SystemData",relPath=os.path.join("private/experiment_config.json"),dataOut="keygroq"):
    configPath=os.path.join(os.getcwd(),relPath)
    with open(configPath, 'r', encoding='utf-8') as file:
        config = json.load(file)[nameModel]
    Output= config[dataOut]
    return Output

prompt="""You are an expert in blockchain smart contract analysis. You will be provided with the source code of a smart contract. Your task is to perform a comprehensive analysis of the contract, covering the following aspects:

Functionality Analysis:

- Identify and explain the main functions of the contract.
- Describe the general purpose of the contract.
- Analyze how different parts of the code interact.

Sector and Use Classification:

Determine in which funtions used. The funtions you can mention are only [DeFi, NFTs, Governance, Play-to-Earn Games,Prediction Markets,Reward Systems in Content Platforms, Fungible Tokens, Real Asset Tokenization Systems, Self-Sovereign Identity Systems, Credential Verification, Decentralized Reputation Management]

Identification of Potential Issues:

- Review the code for common security vulnerabilities in smart contracts.
- Identify possible inefficiencies in gas usage.
- Point out areas where the code could be improved in terms of clarity or optimization.
- Warn about any practices that could lead to scalability or maintenance issues.

Summary and Recommendations:

- Provide a concise summary of key findings.
- Offer recommendations for improving the contract.

Format a json:

{"Functionality Analysis":[Your detailed analysis here], "Sector and Use Classification":[Your classification and without explanation here],"Identification of Potential Issues":[Your list of potential problems here],"Summary and Recommendations":[Your summary and recommendations here]}


Ensure that your analysis is detailed yet concise, using appropriate technical terminology for smart contracts and blockchain development. If you find anything unusual or innovative in the code, make sure to mention it.

Smart Contract Code to analyze:

%s

Please proceed with your analysis based on the provided code.
Respond only with valid JSON. Do not write an introduction or summary.

"""    


def AnalysisContract(source):
    keygroq=extractConfig(nameModel="SystemData",dataOut="keygroq")
    from groq import Groq
    client = Groq(
    api_key=keygroq,
    )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt%(source),
            }
        ],
        model="llama-3.1-70b-versatile",
    )
    return chat_completion.choices[0].message.content

SD=AnalysisContract(B["source_code"])

In [32]:
json.loads(SD)

{'Functionality Analysis': "This contract is an implementation of the ERC20 token standard, specifically designed to wrap EduChain (EDU) native tokens into ERC20-compliant WEDU tokens. The main functions of the contract are deposit, withdraw, and receiveForSpender. The deposit function allows users to send EDU and receive an equivalent amount of WEDU tokens, while the withdraw function allows users to burn WEDU tokens and receive the equivalent amount of EDU. The receiveForSpender function allows an approved spender to use WEDU tokens on behalf of the sender. The contract uses OpenZeppelin's ERC20 contract as its base contract. The receive function is a fallback function that calls the deposit function when EDU is sent to the contract. The different parts of the code interact through the OpenZeppelin ERC20 contract and the Solidity language's built-in functionality.",
 'Sector and Use Classification': ['DeFi', 'Fungible Tokens'],
 'Identification of Potential Issues': ["The contract is

In [ ]:
from fastapi import FastAPI, Request
from fastapi.responses import HTMLResponse,JSONResponse
from pydantic import BaseModel
from fastapi.staticfiles import StaticFiles
from fastapi import Query, File, UploadFile,HTTPException
from starlette.middleware.cors import CORSMiddleware
import pandas as pd
import requests, json, uuid
from markupsafe import Markup
import numpy as np
from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles
from typing import List, Set, Tuple, Dict
from models.db import db
import private.modelcontract as LLMC
from schema import select_stadistics_index,select_item_block,select_items_tx,select_item_contract,select_item_bachOCS2ARB,select_item_bachARB2ETH,select_item_bachETH2ETH
from config.parameters import recaptcha_site_key, recaptcha_secret_key

: 